In this lecture, we will be studying data summarization, aggregation, data merging and joining, and other important operations in 'pandas'. 

We first study aggregation. When we think about aggregation in R, we think about sapply() or lapply(). In SAS and SQL, we use the group-by clauses. In Python, the groupby() method allows you to group rows of data together and call aggregate functions.

To start with, let's create a pseudo dataset from a dictionary that has three columns:

In [24]:
import numpy as np
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.2f' % x) # keeping 2 digits after decimal for display formatting

In [25]:
data = {'Region':['USA','USA','EU','EU','USA','USA','USA','EU'], 'Company':['GOOG','GOOG', 'GOOG','MSFT','MSFT','MSFT', 'FB','FB'], 'Person':['Bobby','Sam','Charlie','Chelsey', 'Amy','Vanessa','Carl','Sarah'],'Sales':[450,200,120,230,340,124,243,350], 'Cost':[45,10,20,40,30,50,20,90]}
df = pd.DataFrame(data)
print(df)

  Region Company   Person  Sales  Cost
0    USA    GOOG    Bobby    450    45
1    USA    GOOG      Sam    200    10
2     EU    GOOG  Charlie    120    20
3     EU    MSFT  Chelsey    230    40
4    USA    MSFT      Amy    340    30
5    USA    MSFT  Vanessa    124    50
6    USA      FB     Carl    243    20
7     EU      FB    Sarah    350    90


Now we can create a 'DataFrameGroupBy' object and then apply its associated methods. Below are some examples of calling the methods associated with the 'DataFrameGroupBy' objects:

In [26]:
print(type(df.groupby("Company")))
by_comp=df.groupby(["Company"])

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [27]:
print(by_comp.mean())
print(by_comp.std())
print(by_comp.count())
print(type(by_comp.max())) # this is a 'DataFrame'

         Sales  Cost
Company             
FB      296.50 55.00
GOOG    256.67 25.00
MSFT    231.33 40.00
         Sales  Cost
Company             
FB       75.66 49.50
GOOG    172.14 18.03
MSFT    108.01 10.00
         Region  Person  Sales  Cost
Company                             
FB            2       2      2     2
GOOG          3       3      3     3
MSFT          3       3      3     3
<class 'pandas.core.frame.DataFrame'>


There is a describe() method associated with the 'DataFrameGroupBy' objects. This method essentially is equivalent to the summary() function in R, or proc means procedure in SAS. You can transpose the table using the transpose() method too:

In [28]:
by_comp.describe()

Sales                                                   Cost        \
        count   mean    std    min    25%    50%    75%    max count  mean   
Company                                                                      
FB       2.00 296.50  75.66 243.00 269.75 296.50 323.25 350.00  2.00 55.00   
GOOG     3.00 256.67 172.14 120.00 160.00 200.00 325.00 450.00  3.00 25.00   
MSFT     3.00 231.33 108.01 124.00 177.00 230.00 285.00 340.00  3.00 40.00   

                                             
          std   min   25%   50%   75%   max  
Company                                      
FB      49.50 20.00 37.50 55.00 72.50 90.00  
GOOG    18.03 10.00 15.00 20.00 32.50 45.00  
MSFT    10.00 30.00 35.00 40.00 45.00 50.00

In [29]:
by_comp.describe().transpose()

Company         FB   GOOG   MSFT
Sales count   2.00   3.00   3.00
      mean  296.50 256.67 231.33
      std    75.66 172.14 108.01
      min   243.00 120.00 124.00
      25%   269.75 160.00 177.00
      50%   296.50 200.00 230.00
      75%   323.25 325.00 285.00
      max   350.00 450.00 340.00
Cost  count   2.00   3.00   3.00
      mean   55.00  25.00  40.00
      std    49.50  18.03  10.00
      min    20.00  10.00  30.00
      25%    37.50  15.00  35.00
      50%    55.00  20.00  40.00
      75%    72.50  32.50  45.00
      max    90.00  45.00  50.00

In [30]:
by_comp.describe().transpose()['FB'] # subsetting only on one column

Sales  count     2.00
       mean    296.50
       std      75.66
       min     243.00
       25%     269.75
       50%     296.50
       75%     323.25
       max     350.00
Cost   count     2.00
       mean     55.00
       std      49.50
       min      20.00
       25%      37.50
       50%      55.00
       75%      72.50
       max      90.00
Name: FB, dtype: float64

To perform the group-by algorithm using more than one key, simply use the brackets '[]':

In [31]:
by_comp=df.groupby(["Company", "Region"])['Cost']
print(by_comp.max())
print(type(by_comp.max())) # this is a 'Series' object because we did a subsetting condition on 'Cost'

Company  Region
FB       EU        90
         USA       20
GOOG     EU        20
         USA       45
MSFT     EU        40
         USA       50
Name: Cost, dtype: int64
<class 'pandas.core.series.Series'>


Sometimes we want to have a group-by analysis based on different functions. For example, we can look at the count, maximum, minimum etc. of the target variable. Suppose in our example, we want to look at both minimum and maximum of the Sales variable by Region:

In [33]:
basic_df=df.groupby(['Region'])['Sales'].agg(["min", "max"])
basic_df

,min,max
Region,,
EU,120,350
USA,124,450


Compare the code above with below. Notice that the reset_index() method does the magic of resetting the index for the dataset below so that it's not 'multi-indexed' as above:

In [35]:
basic_df=df.groupby(['Region'])['Sales'].agg(["min", "max"]).reset_index()
basic_df

,Region,min,max
0,EU,120,350
1,USA,124,450


We can also apply certain functions (such as lambda expressions) to the variable rather than the standard functions such as 'min' and 'max':

In [36]:
new_df=df.groupby(['Region']).agg(min=("Sales", "min"),
                                  max=("Sales", "max"),
                                  mean=("Sales", "mean"),
                                  count=("Sales", "count"),
                                  std=("Sales", "std"),
                                  percentile_25 = ('Sales', lambda x: x.quantile(0.25)),
                                  percentile_50 = ('Sales', lambda x: x.quantile(0.50)),
                                  percentile_75 = ('Sales', lambda x: x.quantile(0.75))
                                  ).reset_index()
new_df

,Region,min,max,mean,count,std,percentile_25,percentile_50,percentile_75
0,EU,120,350,233.33,3,115.04,175.00,230.00,290.00
1,USA,124,450,271.40,5,126.72,200.00,243.00,340.00


This is equivalene to the code below:

In [37]:
by_region=df.groupby(['Region'])
by_region['Sales'].describe().reset_index()

,Region,count,mean,std,min,25%,50%,75%,max
0,EU,3.00,233.33,115.04,120.00,175.00,230.00,290.00,350.00
1,USA,5.00,271.40,126.72,124.00,200.00,243.00,340.00,450.00


Now let's look at some other examples aggregation, but this time with some missing values:

In [39]:
df_ = pd.DataFrame([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [np.nan, np.nan, np.nan]],
                   columns=['A', 'B', 'C'])
df_

,A,B,C
0,1.00,2.00,3.00
1,4.00,5.00,6.00
2,7.00,8.00,9.00
3,NaN,NaN,NaN


In [40]:
print(df_.agg(['sum', 'min'])) # aggregating across rows, thus the default is to apply any function to each column (axis=0)

        A     B     C
sum 12.00 15.00 18.00
min  1.00  2.00  3.00


In [41]:
df_.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']}) # different aggregation

,A,B
sum,12.00,NaN
min,1.00,2.00
max,NaN,8.00


In [42]:
df_.agg('mean', axis=1) # summing across columns

0   2.00
1   5.00
2   8.00
3    NaN
dtype: float64

Now we study merging, concatenating and joining. As a preparation, we need to create some data first:

In [9]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7]) 
df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11])
items=[df1,df2,df3]
for i in items:
    print(i, '\n')

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3 

    A   B   C   D
4  A4  B4  C4  D4
5  A5  B5  C5  D5
6  A6  B6  C6  D6
7  A7  B7  C7  D7 

      A    B    C    D
8    A8   B8   C8   D8
9    A9   B9   C9   D9
10  A10  B10  C10  D10
11  A11  B11  C11  D11 



We now start from the simpliest example called concatenation. Concatenation in Python basically means stacking. Most of the time we want to do vertical stacking. In other words, we want to combine them without any keys and stack them on top of each other.

Vertical stacking means 'axis=0' for the concat() method, as shown below:

In [10]:
pd.concat([df1,df2,df3], axis=0)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


If 'axis=1' is set for the concat() method, then the stacking is horizontal. In other words, we will stack the columns horizontally and so there will be a lot of null values. Notice that if we concatenate this way, we may have duplicate columns. This type of operations can only be done if the two datasets do not share the same column names.

In [11]:
pd.concat([df1,df2,df3],axis=1)

,A,B,C,D,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,A4,B4,C4,D4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,A5,B5,C5,D5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,A6,B6,C6,D6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,A7,B7,C7,D7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A8,B8,C8,D8
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A9,B9,C9,D9


Now let's talk about inner join, left join, and right joins. First, we create two datasets D1 and D2:

In [12]:
D1 = pd.DataFrame({'Keys': ['K0', 'K1', 'K2', 'K3', 'K4', 'K5'],
                     'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5'],
                     'B': ['B0', 'B1', 'B2', 'B3', 'B4', 'B5']})
   
D2 = pd.DataFrame({'Keys': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3'],
                      'E': ['E0', 'E1', 'E2', 'E3']}) 
print(D1, '\n') # 6*3
print(D2, '\n') # 4*4

    A   B Keys
0  A0  B0   K0
1  A1  B1   K1
2  A2  B2   K2
3  A3  B3   K3
4  A4  B4   K4
5  A5  B5   K5 

    C   D   E Keys
0  C0  D0  E0   K0
1  C1  D1  E1   K1
2  C2  D2  E2   K2
3  C3  D3  E3   K3 



We now study inner join, outer (full) join, left (outer) join and right (outer) join. These are achieved by the merge() method in the 'pandas' library. The most important things to memorize for the merge() method is that no matter how you join the datasets, the final records that are kept in the final data only contain records that are merged on the key variables. So left join here uses only keys from the left data frame. Right join here uses only keys from the right data frame. Outer (full) join here uses keys from both data frames, and lastly, inner join (the default join) keeps records whose keys exist in both datasets. 

In [13]:
pd.merge(D1,D2, how='inner', on='Keys') # this is the same as inner join 'a and b'

,A,B,Keys,C,D,E
0,A0,B0,K0,C0,D0,E0
1,A1,B1,K1,C1,D1,E1
2,A2,B2,K2,C2,D2,E2
3,A3,B3,K3,C3,D3,E3


In [14]:
pd.merge(D1,D2, how='outer', on='Keys') # this is the same as outer full join 'a or b'

,A,B,Keys,C,D,E
0,A0,B0,K0,C0,D0,E0
1,A1,B1,K1,C1,D1,E1
2,A2,B2,K2,C2,D2,E2
3,A3,B3,K3,C3,D3,E3
4,A4,B4,K4,NaN,NaN,NaN
5,A5,B5,K5,NaN,NaN,NaN


In [15]:
pd.merge(D1,D2, how='left', on='Keys') 

,A,B,Keys,C,D,E
0,A0,B0,K0,C0,D0,E0
1,A1,B1,K1,C1,D1,E1
2,A2,B2,K2,C2,D2,E2
3,A3,B3,K3,C3,D3,E3
4,A4,B4,K4,NaN,NaN,NaN
5,A5,B5,K5,NaN,NaN,NaN


In [16]:
pd.merge(D1,D2, how='right', on='Keys') 

,A,B,Keys,C,D,E
0,A0,B0,K0,C0,D0,E0
1,A1,B1,K1,C1,D1,E1
2,A2,B2,K2,C2,D2,E2
3,A3,B3,K3,C3,D3,E3


You can also merge on more than one keys using syntax such as pd.merge(D1,D2, how='right', on=['Key1', 'Key2']). 

When it comes to merging and joining, we absolutely need to talk about dedupping. Joining or merging on duplicate keys in Python can cause a returned 'DataFrame' object that is the multiplication of the row dimensions, which may result in memory overflow. Therefore, it is the user's responsibility to manage duplicate values in keys before joining large 'DataFrames' objects.

To dedup the data, we can use the drop_duplicates() method associated with 'DataFrame' objects. This method contains two key arguments('subset' and 'keep'). The 'subset=' argument indicates that the method only considers certain columns for identifying duplicates (by default we use all of the columns). The 'keep=' argument indicates if there are duplicates, which record we actually keep (first or last):

In [17]:
D = pd.DataFrame({'A' : [1,2,3,1,1], 'B' : [1,2,3,1,1], 'C': [1,5,2,1,9]})
print(D)
D.drop_duplicates(subset=['A','B'], keep='first')

   A  B  C
0  1  1  1
1  2  2  5
2  3  3  2
3  1  1  1
4  1  1  9


,A,B,C
0,1,1,1
1,2,2,5
2,3,3,2


When it comes joining on indices, there is a convenient way of doing it. Using the join() method is a convenient way for combining the columns of two potentially differently-indexed 'DataFrames' into a single resulting 'DataFrame'. The only difference between this method and the merge() is that join() uses indices as keys, whereas the merge() joins on column names. Below are some examples:

In [18]:
D3 = pd.DataFrame({'A': ['A0', 'A1', 'A2'],'B': ['B0', 'B1', 'B2']},index=['K0', 'K1', 'K2']) 
D4 = pd.DataFrame({'C': ['C0', 'C2', 'C3'],'D': ['D0', 'D2', 'D3']},index=['K0', 'K2', 'K3'])
print(D3)
print(D4)

     A   B
K0  A0  B0
K1  A1  B1
K2  A2  B2
     C   D
K0  C0  D0
K2  C2  D2
K3  C3  D3


In [19]:
print(D3.join(D4)) # inner join by default
print(D3.join(D4, how='outer')) # outer join

     A   B    C    D
K0  A0  B0   C0   D0
K1  A1  B1  NaN  NaN
K2  A2  B2   C2   D2
      A    B    C    D
K0   A0   B0   C0   D0
K1   A1   B1  NaN  NaN
K2   A2   B2   C2   D2
K3  NaN  NaN   C3   D3


As you see from the previous examples, the merging and joining in Python does not require sorting like SAS data steps. However, duplicates must be taken care of first. Moreover, it's always nice to know how to sort the data even though sorting sometimes is not required for data merging.

To sort a 'DataFrame' object, simply use the sort_values() method:

In [20]:
D5 = pd.DataFrame({'A' : [1,2,3,1,1], 'B' : [1,2,3,1,1], 'C': [1,5,2,1,9]})
print(D5)
D5.sort_values(by=['B','C'], ascending=[True, False]) #inplace=False by default

   A  B  C
0  1  1  1
1  2  2  5
2  3  3  2
3  1  1  1
4  1  1  9


,A,B,C
4,1,1,9
0,1,1,1
3,1,1,1
1,2,2,5
2,3,3,2


More details of sorting can be found below:

In [21]:
help(D5.sort_values)

Help on method sort_values in module pandas.core.frame:

sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last') method of pandas.core.frame.DataFrame instance
    Sort by the values along either axis
    
    .. versionadded:: 0.17.0
    
    Parameters
    ----------
    by : str or list of str
        Name or list of names which refer to the axis items.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis to direct sorting
    ascending : bool or list of bool, default True
         Sort ascending vs. descending. Specify list for multiple sort
         orders.  If this is a list of bools, must match the length of
         the by.
    inplace : bool, default False
         if True, perform operation in-place
    kind : {'quicksort', 'mergesort', 'heapsort'}, default 'quicksort'
         Choice of sorting algorithm. See also ndarray.np.sort for more
         information.  `mergesort` is the only stable algorithm. For
         DataFrames

Now let's go over some additional operations on 'DataFrame' objects. In explorative data analysis (EDA), once we have a dataset, we often want to know some summary information. For example, we want to know the column and index names, we want to check null values, we want to look at the first few records to have a feeling about what the dataset looks like. Below are some techniques often used in elementary EDA:

In [22]:
o1=df['Company'].unique() # getting the unique value of the company
print(type(o1)) # this is an array
print(o1)

<class 'numpy.ndarray'>
['GOOG' 'MSFT' 'FB']


In [23]:
o2=df['Region'].nunique() # returning the number of unique elements in the object
print(o2)

2


In [24]:
o3=df['Cost'].value_counts(sort=True, ascending=False, dropna=True) # returning objects containing counts of unique values
print(type(o3)) # Series
print(o3)

<class 'pandas.core.series.Series'>
20    2
30    1
45    1
10    1
40    1
90    1
50    1
Name: Cost, dtype: int64


In [25]:
o4=df.columns
o5=df.index
print(o4)
print(o5)
print(type(o4))
print(type(o5))

Index(['Company', 'Cost', 'Person', 'Region', 'Sales'], dtype='object')
RangeIndex(start=0, stop=8, step=1)
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.range.RangeIndex'>


In [26]:
o6=df.isnull()
print(type(o6)) # this is a 'DataFrame' object
o6

<class 'pandas.core.frame.DataFrame'>


,Company,Cost,Person,Region,Sales
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
5,False,False,False,False,False
6,False,False,False,False,False
7,False,False,False,False,False


In [27]:
df.pivot_table(values='Sales',index=['Company', 'Region'],columns=['Person']) 

Person            Amy  Bobby   Carl  Charlie  Chelsey    Sam  Sarah  Vanessa
Company Region                                                              
FB      EU        NaN    NaN    NaN      NaN      NaN    NaN  350.0      NaN
        USA       NaN    NaN  243.0      NaN      NaN    NaN    NaN      NaN
GOOG    EU        NaN    NaN    NaN    120.0      NaN    NaN    NaN      NaN
        USA       NaN  450.0    NaN      NaN      NaN  200.0    NaN      NaN
MSFT    EU        NaN    NaN    NaN      NaN    230.0    NaN    NaN      NaN
        USA     340.0    NaN    NaN      NaN      NaN    NaN    NaN    124.0

Lastly, we will learn how to apply functions to a column of a 'DataFrame' object:

In [28]:
def times2(x):
    return x*2
o7=df['Sales'].apply(times2)
print(type(o7)) # this is a 'Series' object
print(o7)
o8=df['Region'].apply(len)
print(o8)

<class 'pandas.core.series.Series'>
0    900
1    400
2    240
3    460
4    680
5    248
6    486
7    700
Name: Sales, dtype: int64
0    3
1    3
2    2
3    2
4    3
5    3
6    3
7    2
Name: Region, dtype: int64
